# Oscars ETL Data Engineering Project

This notebook extracts, transforms, and loads data about Oscar-winning films from a web source.

In [1]:
import pandas as pd
import requests
import re

## Extract

Extracting data from the web source. The "year" field was already cleaned in this phase as a "pre-transform" measure.

In [2]:
# Function to get the JSON data from the URL
def fetch_json_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Failed to fetch data from {url}, status code: {response.status_code}")

In [3]:
# Function to clean the year field and extract the latest year mentioned
def extract_latest_year(year_text):
    match = re.search(r'(\d{2})(\d{2}) / (\d{2})', year_text)
    if match:
        return int(f"{match.group(1)}{match.group(3)}")
    else:
        # If the pattern does not match, fall back to finding the latest year
        matches = re.findall(r'\d{4}', year_text)
        if matches:
            return max(map(int, matches))
    return None

In [4]:
# Function to get the budget from the film's detail URL
def fetch_budget(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            film_data = response.json()
            return film_data.get('Budget', 0)
        else:
            return 0
    except:
        return 0

In [5]:
# URL of the JSON file
url = "http://oscars.yipitdata.com/"

# Fetch data
data = fetch_json_data(url)

# Extract the required information
films_data = []
for year_data in data['results']:
    year = extract_latest_year(year_data.get('year', ''))
    for film in year_data['films']:
        detail_url = film.get('Detail URL')
        budget = fetch_budget(detail_url)
        film_info = {
            'film': film.get('Film'),
            'year': year,
            'wiki_url': film.get('Wiki URL'),
            'oscar_winner': film.get('Winner'),
            'budget': budget
        }
        films_data.append(film_info)

## Transform

Transformation was mainly focused on the "budget" field.

In [ ]:
# Convert to Pandas DataFrame
films_df = pd.DataFrame(films_data)

In [6]:
def clean_budget(budget):
    if isinstance(budget, (int, float)):
        return budget
    
    # Remove content inside brackets and parentheses
    budget = re.sub(r'\[.*?\]|\(.*?\)', '', budget).strip()
    
    # Get the highest value in case of a range of values
    # if '–' in budget or '-' in budget:
    #     budget = budget.replace('–', '-')
    #     parts = budget.split('-')
    #     if not parts[1].startswith('$'):
    #         budget = "$" + parts[1].strip()
    #     else:
    #         budget = parts[1].strip()
            
    # Check for range and set to $0 if true
    if '–' in budget or '-' in budget:
        budget = '0'
    
    # Replace $ with US$ if it starts with $
    if budget.startswith('$'):
        budget = budget.replace('$', 'US$', 1)
        
    # Replace USD$ with US$
    budget = budget.replace('USD$', 'US$')
    
    # Detect and replace patterns like ".xxx.xxx" with ",xxx,xxx"
    budget = re.sub(r'(\d)\.(\d{3})\.(\d{3})', r'\1,\2,\3', budget)
    
    # Convert millions to numbers for different currencies
    match = re.match(r'(US\$|£|€)\s*([\d\.]+)\s*million', budget, re.IGNORECASE)
    if match:
        currency = match.group(1)
        amount = float(match.group(2))
        budget = f'{currency} {amount * 1_000_000:,.0f}'
        
    # Remove in-between spaces
    budget = budget.replace(' ', '')
    
    return budget

# Apply the function to the budget column
films_df['budget'] = films_df['budget'].apply(clean_budget)

In [8]:
# Function to get exchange rates
def get_exchange_rate(base_currency, target_currency):
    api_key = 'your_api_key_here'
    url = f'https://v6.exchangerate-api.com/v6/{api_key}/latest/{base_currency}'
    response = requests.get(url)
    data = response.json()
    return data['conversion_rates'][target_currency]

# Function to convert currencies to USD
def convert_to_usd(budget):
    # Ensure budget is a string
    budget = str(budget)
    
    # Set the 0 string budgets to int
    if budget == '0':
        return 0
    
    # Extract currency and amount
    match = re.match(r'(US\$|£|₤|€)\s*([\d,\.]+)', budget)
    if match:
        currency = match.group(1)
        amount = float(match.group(2).replace(',', ''))
        
        if currency in ['£', '₤']:
            # Convert GBP to USD
            rate = get_exchange_rate('GBP', 'USD')
            amount_in_usd = amount * rate
        elif currency == '€':
            # Convert EUR to USD
            rate = get_exchange_rate('EUR', 'USD')
            amount_in_usd = amount * rate
        else:
            # Already in USD
            amount_in_usd = amount
        
        return int(amount_in_usd)
    
    # Return 0 if no match
    return 0

# Apply the conversion function to create a new column
films_df['budget_in_usd'] = films_df['budget'].apply(convert_to_usd)

## Load

Loading data into CSV format locally without the index.

In [9]:
films_df.dtypes

film             object
year              int64
wiki_url         object
oscar_winner       bool
budget           object
budget_in_usd     int64
dtype: object

In [10]:
# Load the file in CSV format
films_df.to_csv('films.csv', index=False)